In [70]:
import requests
import csv 
from collections import Counter
from pandas import *


In [85]:
listoffiles=[]
listoffilmnames=[]

def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s=requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostchecked.append(row[3])
        toplist.append(row[5])        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    d = {   'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(d)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

now 'loginandgetcsv' will save a .csv in the parent directory called *username*_checklist.csv

In [86]:
listmaker('Haessar')

,most_checked,title,top_lists
0,36182,Platoon,17
1,506,Tracks,0
2,3024,Norbit,0
3,64,Alive Inside,0
4,57430,Mr. & Mrs. Smith,2
5,57278,Rocky,19
6,6545,22 Jump Street,1
7,58998,Star Wars: Episode III - Revenge of the Sith,4
8,35835,The Graduate,25
9,28354,Some Like It Hot,27


In [27]:
username1='Haessar'
username2='hermesmovies'
username3='James88'
password=''

In [28]:
csvgetter(username1,password,username2,username3)

There are 9 users in the database: 
_checklist.csv
_checklist.csv
Haessar_checklist.csv
hermesmovie_checklist.csv
James88_checklist.csv
Haessar_checklist.csv
hermesmovies_checklist.csv
James88_checklist.csv
Haessar_checklist.csv


In [29]:
willlist=listmaker(username2)
davelist=listmaker(username1)
jameslist=listmaker(username3)

['War of the Worlds', 'Signal to Noise: Atari: Game Over', 'A Serious Man', 'Casino', 'Million Dollar Baby', 'It Follows', 'The Princess Bride', "Devil's Advocate", 'The Kids Are All Right', 'The English Patient', 'Lilo & Stitch', 'Swingers', 'A Million Ways to Die in the West', 'The Birds', 'I Am Legend', "Schindler's List", 'Orphan', 'Presto', 'Edge of Tomorrow', 'Braveheart', '2001: A Space Odyssey', 'In a World...', 'Deux jours, une nuit', 'Horns', 'Piled Higher and Deeper', 'The Nutty Professor', 'Still Alice', 'Oliver!', 'Nightcrawler', 'Inherent Vice', 'The Warriors', 'Selma', 'Ghost Town', 'The Borrowers', 'Chef', 'The Guilt Trip', 'Foxcatcher', 'A Single Life', 'Me and My Moulton', 'Footprints', 'Feast', 'Duet', 'The Dam Keeper', 'The Bigger Picture', 'Cool Runnings', 'Evolution', 'American Sniper', 'Attack the Block', 'How to Train Your Dragon 2', 'The Boxtrolls', 'Harry Brown', 'Big Hero 6', 'Kaguyahime no monogatari', 'Frank', 'Catfish', 'Grizzly Man', 'Birdman or (The Unex

In [19]:
#this is working on a function that will do everything
def superfunction(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    multiname(username1,username2,username3)

In [30]:
superlist=[]
davewillshared=[]
davejamesshared=[]
willjamesshared=[]
superlist.extend(willlist+davelist+jameslist)
davewillshared.extend(willlist+davelist)
davejamesshared.extend(davelist+jameslist)
willjamesshared.extend(willlist+jameslist)

In [31]:
nodups=list(set(superlist)) 
davewillshared_nodups=list(set(davewillshared)) 
davejamesshared_nodups=list(set(davejamesshared)) 
willjamesshared_nodups=list(set(willjamesshared)) 

In [34]:
len(davewillshared)

1748

In [66]:
def process_list(data,others):
    nrec=[k for k,v in Counter(others).items() if v>1]
    user_all
    return(nrec)

In [52]:
others=willlist+davelist

In [67]:
test=process_list(jameslist,others)